# Evolver Loop 4 Analysis

Analyzing the current state and planning the next experiment.

## Key Findings:
1. SA found NO improvements - baseline is at extremely strong local optimum
2. Any perturbation creates overlaps
3. Need fundamentally different approach: EXHAUSTIVE SEARCH for small N

In [1]:
import sys
sys.path.insert(0, '/home/code')
import pandas as pd
import numpy as np
import json

# Load baseline
baseline_df = pd.read_csv('/home/code/experiments/001_valid_baseline/submission.csv')

# Parse baseline
from collections import defaultdict
configs = defaultdict(list)
for _, row in baseline_df.iterrows():
    parts = row['id'].split('_')
    n = int(parts[0])
    x = float(str(row['x']).replace('s', ''))
    y = float(str(row['y']).replace('s', ''))
    deg = float(str(row['deg']).replace('s', ''))
    configs[n].append((x, y, deg))

print("Baseline configurations loaded")
print(f"Total N values: {len(configs)}")
print(f"\nN=1 config: {configs[1]}")
print(f"N=2 config: {configs[2]}")
print(f"N=3 config: {configs[3]}")

Baseline configurations loaded
Total N values: 200

N=1 config: [(-48.196086194214246, 58.770984615214225, 45.0)]
N=2 config: [(0.1540970696213643, -0.03854074269478543, 203.62937773065684), (-0.1540970696213643, -0.5614592573052146, 23.629377730656792)]
N=3 config: [(1.123655816140301, 0.781101815992563, 111.125132292893), (1.23405569584216, 1.275999500663759, 66.370622269343), (0.641714640229075, 1.180458566613381, 155.13405193710082)]


In [2]:
# Calculate scores for small N
from code.tree_geometry import calculate_score, calculate_bbox

print("Baseline scores for small N:")
for n in range(1, 11):
    score = calculate_score(configs[n])
    print(f"  N={n}: {score:.6f}")

Baseline scores for small N:


  N=1: 0.661250
  N=2: 0.450779
  N=3: 0.434745
  N=4: 0.416545
  N=5: 0.416850
  N=6: 0.399610
  N=7: 0.399897
  N=8: 0.385407
  N=9: 0.387415
  N=10: 0.376630


In [3]:
# Analyze N=2 configuration in detail
print("\n=== N=2 Configuration Analysis ===")
tree1 = configs[2][0]
tree2 = configs[2][1]
print(f"Tree 1: x={tree1[0]:.6f}, y={tree1[1]:.6f}, angle={tree1[2]:.6f}")
print(f"Tree 2: x={tree2[0]:.6f}, y={tree2[1]:.6f}, angle={tree2[2]:.6f}")

# Calculate relative position
dx = tree2[0] - tree1[0]
dy = tree2[1] - tree1[1]
print(f"\nRelative position: dx={dx:.6f}, dy={dy:.6f}")
print(f"Angle difference: {tree2[2] - tree1[2]:.6f} degrees")


=== N=2 Configuration Analysis ===
Tree 1: x=0.154097, y=-0.038541, angle=203.629378
Tree 2: x=-0.154097, y=-0.561459, angle=23.629378

Relative position: dx=-0.308194, dy=-0.522919
Angle difference: -180.000000 degrees


In [4]:
# Key insight: For N=2, we can do exhaustive search
# The search space is:
# - Tree 1 angle: 0-360 degrees
# - Tree 2 angle: 0-360 degrees  
# - Tree 2 position: determined by bottom-left placement given angles

# At 1 degree resolution: 360 * 360 = 129,600 combinations
# At 0.5 degree resolution: 720 * 720 = 518,400 combinations
# At 0.1 degree resolution: 3600 * 3600 = 12.96M combinations

print("Search space for N=2:")
print(f"  1.0° resolution: {360*360:,} combinations")
print(f"  0.5° resolution: {720*720:,} combinations")
print(f"  0.1° resolution: {3600*3600:,} combinations")
print(f"\nWith Numba, we can easily search 1M+ combinations per second")

Search space for N=2:
  1.0° resolution: 129,600 combinations
  0.5° resolution: 518,400 combinations
  0.1° resolution: 12,960,000 combinations

With Numba, we can easily search 1M+ combinations per second


In [5]:
# The key challenge is: given two angles, how to find the best placement?
# 
# For N=2, we can:
# 1. Fix tree 1 at origin with angle a1
# 2. For tree 2 with angle a2, find the position that minimizes bbox
#
# The optimal position for tree 2 is where it "touches" tree 1
# without overlapping, and minimizes the bounding box.
#
# This is a 2D optimization problem for each (a1, a2) pair.
# We can use a grid search or gradient descent.

print("Strategy for exhaustive N=2 search:")
print("1. For each angle pair (a1, a2):")
print("   a. Place tree 1 at origin")
print("   b. Find optimal position for tree 2 (minimize bbox, no overlap)")
print("   c. Record score")
print("2. Return best configuration")
print("\nThe position search can be done with:")
print("   - Grid search around tree 1")
print("   - Bottom-left heuristic")
print("   - Gradient descent from multiple starting points")

Strategy for exhaustive N=2 search:
1. For each angle pair (a1, a2):
   a. Place tree 1 at origin
   b. Find optimal position for tree 2 (minimize bbox, no overlap)
   c. Record score
2. Return best configuration

The position search can be done with:
   - Grid search around tree 1
   - Bottom-left heuristic
   - Gradient descent from multiple starting points
